In [ ]:
from pathlib import Path
import logging
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.strtree import STRtree
from shapely.geometry import Point

TOWER_FILE        = Path("data/slovenia_towers.parquet")
MAX_RADIUS_M      = 300    # accept towers within this radius

# ─────────────────────── towers (once) ────────────────────────────
tower_df  = pd.read_parquet(TOWER_FILE)
tower_gdf = gpd.GeoDataFrame(
    tower_df,
    geometry=gpd.points_from_xy(tower_df.LON, tower_df.LAT),
    crs="EPSG:4326"
).to_crs("EPSG:3857")

tower_gdf["tower_id"] = np.arange(len(tower_gdf), dtype="int32")

# ─────────────────── helper: add nearest tower ────────────────────
def add_tower_id(df: pd.DataFrame,
                 max_radius_m: float = MAX_RADIUS_M) -> pd.DataFrame:
    # build point GeoDataFrame in metres
    pts = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.lon, df.lat),
        crs="EPSG:4326"
    ).to_crs("EPSG:3857")

    joined = gpd.sjoin_nearest(
        pts,
        tower_gdf[["tower_id", "geometry"]],
        how="left",
        max_distance=max_radius_m,
        distance_col="tower_dist_m")

    # ── collapse possible duplicates (one row per point) ──────────
    first = (joined
             .sort_index()                       # keep stable order
             .groupby(level=0)                   # original point index
             .first())

    df["tower_id"]     = first["tower_id"].to_numpy()
    df["tower_dist_m"] = first["tower_dist_m"].to_numpy()
    df["tower_ok"]     = first["tower_id"].notna()

    return df